### Уважаемый ментор! Всё пока в сыром виде, сегодня-завтра - доработаю.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
df = pd.read_csv('main_task.csv')
df.info() 
 
def fill_na(column):
  city_list = df.City.unique()
  for city in city_list:
    mean_price = df[column][df['City'] == city].mean()
    df[column][df['City'] == city] = df[column][df['City'] == city].fillna(mean_price)
  return df
 
df.Reviews[0]

In [23]:
 
df
#type(data['Cuisine Style'][0])
#df['Price Range'][df['Price Range']=='$$ - $$$']#.unique()
#len(df.City.unique())
#len(df['Cuisine Style'].unique())
#df['Cuisine Style'].value_counts()
cus_index = ['cuisine', 'count'] 
cuisines = [] #pd.DataFrame() #columns=cus_index) 
cuis_dict = {} 
cuis_count = [] 
 
for index in df.index:#[:1000]:
  string = df['Cuisine Style'][index]
  if str(string) != 'nan':
    i=0
    j=0
    #cuis = []  
    #print(string) 
    #cuisines = cuisines.append(pd.Series([0 for i in range(len(cuisines.columns))], index=cuisines.columns), ignore_index=True)
    cuisines.append([0 for x in range(len(cuis_dict))]) 
    while i<len(string)-1:
      #print(i)
      cuisine = string[string.find("'",i)+1:string.find("'",string.find("'",i)+1)]
      #if len(cuisines[cuisines['cuisine']==cuisine])>0:
        #cuisines['count'][cuisines[cuisines['cuisine']==cuisine].index] +=1
      #else:
        #cuisines = cuisines.append(pd.Series([cuisine, 1], index=cus_index), ignore_index=True) 
      #cuisines.setdefault(cuisine, 0
      if cuisine not in cuis_dict.keys():
        cuis_dict[cuisine] = len(cuis_dict)
        cuisines[index].append(1) 
        #cuisines[cuisine] = 0
        #cuisines.iloc[-1] = cuisines.iloc[-1]
      else:
        cuisines[index][cuis_dict[cuisine]] = 1
      #cuisines[cuisine].iloc[-1] = 1
      i = string.find("'",string.find("'",i)+1)+1
      #i=len(string)
      j += 1
      #print(i) 
    cuis_count.append(j) 
    #cuisines.append(cuis)
  else:
    cuis_count.append(None)
    cuisines.append([None for x in range(len(cuis_dict))])  
    #cuisines.append(None)
    #cuisines = cuisines.append(pd.Series([None for i in range(len(cuisines.columns))], index=cuisines.columns), ignore_index=True)
#df['Cuisine Style'] = cuisines
#df['cuis_count'] = cuis_count
#fill_na('cuis_count') 
#len_cuis = len(cuis_dict) 
#for index in range(len(cuisines)):
'''i = 0 
for row in cuisines:
  if str(row) != 'None':
    cuisines[i] = row + [0 for i in range(len(cuis_dict) - len(row))] 
   # print(i) 
  i += 1''' 
cuisines = pd.DataFrame(cuisines, columns = list(cuis_dict.keys())) #[:15]) 
cuisines = pd.concat([df['Cuisine Style'], cuisines], axis=1)
cuisines = cuisines.dropna(how='all')#pd.get_dummies(cuisines.keys())
cuisines = cuisines.drop(['Cuisine Style'], axis=1) 
cuisines = cuisines.fillna(0)
cuis_undex = cuisines.index
cuisines = dummy(cuisines) 
cuisines = pd.DataFrame({'cuis_norm':cuisines}, index=cuis_undex)
df = pd.concat([df,cuisines], axis=1)
df = fill_na('cuis_norm') 
df['cuis_count'] = cuis_count
df = fill_na('cuis_count')
#df = pd.concat([df,cuisines], axis=1)
#cuisines = 
#df.merge(cuisines, on=index, how='left')
#cuisines 
#cuis_undex
df
#cuisines=pd.DataFrame({'cuisine':cuisines.keys(),'count':cuisines.values()})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,time_change,len_reviews,city_norm,cuis_norm,cuis_count
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,41.000000,72.000000,-0.004789,-0.001971,3.000000
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,382.000000,60.000000,0.004139,0.001242,3.184071
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,2.000000,59.000000,-0.005239,0.006971,7.000000
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,186.632581,64.611047,-0.001717,0.000044,2.941958
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,272.000000,97.000000,0.003496,-0.000183,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,34.000000,89.000000,-0.001669,-0.001765,4.000000
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,9.000000,83.000000,-0.004789,0.000695,5.000000
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,3127.000000,93.000000,0.004139,-0.001963,2.000000
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,23.000000,72.000000,0.005095,0.005592,5.000000


In [ ]:
cuisine= 'jiik'
cuisines = pd.DataFrame()
#cuis_list = [0 for i in range(cuisines.columns)]
#if cuisine in cuisines.columns:
  
#else:
#
cuisines = cuisines.append(pd.Series([0 for i in range(len(cuisines.columns))], index=cuisines.columns), ignore_index=True) 
cuisines[cuisine] = 0
cuisines['67'] = 0
cuisines = cuisines.append(pd.Series([1,0], index=cuisines.columns), ignore_index=True) 
cuisines['gh7'] = 0
cuisines.iloc[-1] = 0
cuisines.iloc[-1]['67']=6
cuisines.iloc[-1]['gh7']=6
cuisines['gyu'] = 0
cuisines.iloc[-1] = cuisines.iloc[-1]
cuisines.iloc[-1]['gyu']=7
cuisines = cuisines.append(pd.Series([0 for i in range(len(cuisines.columns))], index=cuisines.columns), ignore_index=True) 
cuisines['ggh'] = 0
cuisines.iloc[-1] =cuisines.iloc[-1] 
cuisines.iloc[-1]['ggh']=7
cuisines = cuisines.append(pd.Series([None for i in range(len(cuisines.columns))], index=cuisines.columns), ignore_index=True)
cuisines = cuisines.append(pd.Series([0 for i in range(len(cuisines.columns))], index=cuisines.columns), ignore_index=True)
cuisine='Japanese' 
if cuisine not in cuisines.columns:
 cuisines[cuisine] = 0#[0 for i in range(len(cuisines.columns)-1)]+[1]
 cuisines.iloc[-1] = cuisines.iloc[-1]
  
cuisines['Japanese'].iloc[-1] = 1
cuisines=[] 
#cuisines.append([])
cuisines[0]#[0]
#[0 for i in range(len(cuisines.columns)) if i < len(cuisines.columns) else 0]
#[i for i in range(1) if i<1]+[1]
#cuis_list[3]
#list(cuisines.columns).loc[67]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


IndexError: ignored

In [20]:
df.Reviews[1]
pattern_time = re.compile('\d\d\/\d\d\/\d+')
pattern_text = re.compile('[A-Za-z ]+')
t_index = ['time1', 'time2']
time1 = []
time2 = [] 
len_text = [] 
 
for i in df.index:
  t_values = pd.to_datetime(pattern_time.findall(df.Reviews[i])) 
  if len(df.Reviews[i]) != 8:
    len_text.append(len(str(pattern_text.findall(df.Reviews[i]))))
  else:
    len_text.append(None) 
  
  if len(t_values) == 0:
    #time = time.append(pd.Series([None, None], index=t_index), ignore_index=True)
    time1.append(None)
    time2.append(None) 
  elif len(t_values) == 1:
    #time = time.append(pd.Series([t_values, None], index=t_index), ignore_index=True)
    time1.append(t_values[0])
    time2.append(None) 
  else:
    time1.append(t_values[0])
    time2.append(t_values[1]) 
    #time = time.append(pd.Series(t_values, index=t_index), ignore_index=True)
    #print(pattern.findall(df.Reviews[i])) 
#time = pd.to_datetime(time[0]) 
#time1
time = pd.DataFrame({'time1':time1, 'time2':time2})  
time#.head(25)
#df.Reviews[16]
#print(time.time1.min())
time.time2.min()
df['time_change'] = abs(time.time1-time.time2)
#time = pd.concat([time,time.time_change],axis=1)
#time.head(50)
#time.time_change.max()  
#df['time_change'] = [int(i.days) for i in df['time_change'] if str(i.days) != 'nan'] 
#df['time_change'].day
#for day in df['time_change']:
  #if 
df['time_change'] = df['time_change'].apply(lambda x: int(x.days) if str(x.days) != 'nan' else None) 
df = fill_na('time_change') 
df['len_reviews'] = len_text
df = fill_na('len_reviews')
df
#df.Reviews.apply(lambda x: len(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,time_change,len_reviews
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,41.000000,72.000000
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,382.000000,60.000000
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,2.000000,59.000000
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,186.632581,64.611047
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,272.000000,97.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,34.000000,89.000000
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,9.000000,83.000000
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,3127.000000,93.000000
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,23.000000,72.000000


In [ ]:
pattern = re.compile('')

In [24]:
#[[df.City], [i[30:] for i in df.URL_TA]]
#df['Price Range'].unique()
#import warnings; warnings.simplefilter('ignore')
#df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/main_task.csv')
price_dict = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price Range'] = df['Price Range'].replace(to_replace=price_dict)
#& ]#.mean()
 
#mode.chained_assignment=None
fill_na('Price Range') 
df#.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,time_change,len_reviews,city_norm,cuis_norm,cuis_count
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.000000,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,41.000000,72.000000,-0.004789,-0.001971,3.000000
1,id_1535,Stockholm,NaN,1537.0,4.0,1.943182,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,382.000000,60.000000,0.004139,0.001242,3.184071
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.000000,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,2.000000,59.000000,-0.005239,0.006971,7.000000
3,id_3456,Berlin,NaN,3458.0,5.0,1.754799,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,186.632581,64.611047,-0.001717,0.000044,2.941958
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.000000,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,272.000000,97.000000,0.003496,-0.000183,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2.000000,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,34.000000,89.000000,-0.001669,-0.001765,4.000000
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2.000000,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,9.000000,83.000000,-0.004789,0.000695,5.000000
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,1.943182,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,3127.000000,93.000000,0.004139,-0.001963,2.000000
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2.000000,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,23.000000,72.000000,0.005095,0.005592,5.000000


In [22]:
def dummy(column):
  #column = pd.get_dummies(column)
  column = column.apply(lambda x: x-x.mean())#.head(60)
  column = column.apply(lambda x: x/np.linalg.norm(x))
  column = [column.iloc[x].sum() for x in range(len(column))] # = city.apply(lambda x: x/np.linalg.norm(x))
  column = column / np.linalg.norm(column)
  return column
df['city_norm'] = dummy(pd.get_dummies(df.City)) 
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,time_change,len_reviews,city_norm
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,41.000000,72.000000,-0.004789
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,382.000000,60.000000,0.004139
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,2.000000,59.000000,-0.005239
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,186.632581,64.611047,-0.001717
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,272.000000,97.000000,0.003496
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,34.000000,89.000000,-0.001669
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,9.000000,83.000000,-0.004789
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,3127.000000,93.000000,0.004139
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,23.000000,72.000000,0.005095


In [25]:
df = fill_na('Number of Reviews') 
df.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        40000 non-null  float64
 6   Number of Reviews  40000 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
 10  time_change        40000 non-null  float64
 11  len_reviews        40000 non-null  float64
 12  city_norm          40000 non-null  float64
 13  cuis_norm          40000 non-null  float64
 14  cuis_count         40000 non-null  float64
dtypes: float64(9), object(6)
memory usage: 6.1+ MB


In [29]:
df[['Rating', 'Ranking', 'Price Range', 'Number of Reviews', 'cuis_norm', 'cuis_count', 'city_norm', 'time_change', 'len_reviews']].corr()

,Rating,Ranking,Price Range,Number of Reviews,cuis_norm,cuis_count,city_norm,time_change,len_reviews
Rating,1.000000,-0.368371,-0.020994,0.029263,0.093923,0.123747,0.053201,-0.079250,0.013089
Ranking,-0.368371,1.000000,-0.071661,-0.210819,-0.164301,-0.294276,-0.595952,0.122070,-0.085885
Price Range,-0.020994,-0.071661,1.000000,0.162652,0.043569,0.135922,0.011194,-0.053349,0.059027
Number of Reviews,0.029263,-0.210819,0.162652,1.000000,0.141021,0.346193,-0.046171,-0.179010,0.076447
cuis_norm,0.093923,-0.164301,0.043569,0.141021,1.000000,0.666415,0.113152,-0.141608,0.058716
cuis_count,0.123747,-0.294276,0.135922,0.346193,0.666415,1.000000,0.081964,-0.253378,0.114083
city_norm,0.053201,-0.595952,0.011194,-0.046171,0.113152,0.081964,1.000000,-0.002517,0.013284
time_change,-0.079250,0.122070,-0.053349,-0.179010,-0.141608,-0.253378,-0.002517,1.000000,-0.000094
len_reviews,0.013089,-0.085885,0.059027,0.076447,0.058716,0.114083,0.013284,-0.000094,1.000000


In [35]:
X = df[['Ranking', 'Price Range', 'Number of Reviews', 'cuis_norm', 'cuis_count', 'city_norm', 'time_change', 'len_reviews']]
y = df['Rating']  
#df_mean = df['Number of Reviews'].mean()
#df['Number of Reviews']=df['Number of Reviews'].fillna(df_mean) 
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
  
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.217362
